# Preparando o ambiente

In [1]:
from modules.utils import prepare_data_filesystem

prepare_data_filesystem()

[08/04/2024 23:04:52] Tentando criar o diretório data/raw.
[08/04/2024 23:04:52] Diretório data/raw criado com sucesso!!
[08/04/2024 23:04:52] Tentando criar o diretório data/bronze.
[08/04/2024 23:04:52] Diretório data/bronze criado com sucesso!!
[08/04/2024 23:04:52] Tentando criar o diretório data/silver.
[08/04/2024 23:04:52] Diretório data/silver criado com sucesso!!
[08/04/2024 23:04:52] Tentando criar o diretório data/gold.
[08/04/2024 23:04:52] Diretório data/gold criado com sucesso!!


# Programando as execuções

## Execução Geral

Importando as bibliotecas

In [4]:
from modules.etl import landing_step, bronze_step,silver_step,gold_step
from modules.utils import range_de_hora_passado
import time

Execução

In [ ]:
while True:

    inicio,fim = range_de_hora_passado()
    
    # Seleção do tempo de rodagem do código a cada 1h para emissão do arquivo para landing zone
    if fim[-5:] == "00:01": #é considerado 1 segundo a mais para rodar, pois garante a requisição correta quando for as 00H00
        landing_step(query="dengue",languague="pt", inicio=inicio, fim=fim)
        time.sleep(10) 

    # Transforma 1x ao dia todo o arquivo bronze em um parquet diario e também adiciona esse parquet no arquivo silver
    if fim[-8:] == "00:00:30":
        bronze_step()
        time.sleep(10)
        
        silver_step()
        time.sleep(10)

        gold_step()
        time.sleep(10)


## Landing Step

Criando uma base de dados para termos histórico

In [5]:
from modules.etl import landing_step
from datetime import datetime, timedelta

for i in range(0,50):
    inicio = (datetime.now() - timedelta(days=i)).replace(hour=0,minute=0,second=0).strftime(format="%Y-%m-%dT%H:%M:%S")
    fim = (datetime.now() - timedelta(days=i)).replace(hour=23,minute=59,second=59).strftime(format="%Y-%m-%dT%H:%M:%S")
    landing_step(query="dengue",languague="pt",inicio=inicio,fim=fim)

[06/04/2024 00:45:53] FIST STEP (FOR EACH 1 HOUR)
[06/04/2024 00:45:53] Extracting...
[06/04/2024 00:45:53] Construindo os dados a serem enviados na requisição
[06/04/2024 00:45:53] Enviando a requisição
[06/04/2024 00:45:54] Requisição feita com sucesso!!
[06/04/2024 00:45:54] Verificando se é necessárioa mais de uma página
[06/04/2024 00:45:54] Mais de uma página não é necessaŕio! Só um json será salvo!
[06/04/2024 00:45:54] Não há novos artigos no periodo de 2024-04-06T00:00:00 a 2024-04-06T23:59:59
[06/04/2024 00:45:54] FIST STEP (FOR EACH 1 HOUR)
[06/04/2024 00:45:54] Extracting...
[06/04/2024 00:45:54] Construindo os dados a serem enviados na requisição
[06/04/2024 00:45:54] Enviando a requisição
[06/04/2024 00:45:54] Requisição feita com sucesso!!
[06/04/2024 00:45:54] Verificando se é necessárioa mais de uma página
[06/04/2024 00:45:54] Mais de uma página não é necessaŕio! Só um json será salvo!
[06/04/2024 00:45:54] Saving response in data/raw
[06/04/2024 00:45:54] FIST STEP (

## Bronze Step

In [3]:
from modules.etl import bronze_step

bronze_step()

[06/04/2024 00:45:08] Listando os jsons que estão no diretório data/raw
[06/04/2024 00:45:08] Verificando se o diretório data/aw está vazio
[06/04/2024 00:45:08] Criando uma lsita de objetos json
[06/04/2024 00:45:08] Transformando-os em dicionários Python
[06/04/2024 00:45:08] Separando somente a key articles
[06/04/2024 00:45:08] Transformando em dataframes
[06/04/2024 00:45:08] COncatenando os dataframes
[06/04/2024 00:45:08] Inserindo a colunad 'load_date'
[06/04/2024 00:45:08] Verificando se já tem um arquivo bronze


## Silver Step

In [7]:
from modules.etl import silver_step

silver_step()

d:\Documentos\Cursos\Tecnologia\ADA\Extração_de_Dados\ada_extracao_dados2\modules\etl.py:172: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_silver['author'][authorNoneOrNull] = 'Não Informado'
d:\Documentos\Cursos\Tecnologia\ADA\Extração_

In [42]:
# import pandas as pd

# df = pd.read_parquet("data/processed/bronze.parquet")

# # Tratando os tipos de variáveis
# df['publishedAt'] = pd.to_datetime(df['publishedAt']) # datetime
# df['load_date'] = pd.to_datetime(df['load_date']) # datetime
# df['source'] = df['source'].apply(lambda x:x.get("name").lower()) # Trabalhando na coluna source
# df['source'] = df['source'].astype("category") # Categorico
# df = df[df['content']!="[Removed]"] # Retirando artigos que tiveram o conteúdo removido
# df.drop_duplicates(inplace=True) # Retirando registros duplicados

## Gold Step

In [8]:
from modules.etl import gold_step

gold_step()